In [1]:
import itertools

In [2]:
    not_available = ['E017-H3K4me3', 'E017-H3K4me2', 'E017-CTCF', 'E017-H3K9ac']
    exclude_list = [False]
    for i in range(1, 5):
        exclude_list += [j for j in itertools.combinations(not_available, i)]

In [13]:
exclude_list

[False,
 ('E017-H3K4me3',),
 ('E017-H3K4me2',),
 ('E017-CTCF',),
 ('E017-H3K9ac',),
 ('E017-H3K4me3', 'E017-H3K4me2'),
 ('E017-H3K4me3', 'E017-CTCF'),
 ('E017-H3K4me3', 'E017-H3K9ac'),
 ('E017-H3K4me2', 'E017-CTCF'),
 ('E017-H3K4me2', 'E017-H3K9ac'),
 ('E017-CTCF', 'E017-H3K9ac'),
 ('E017-H3K4me3', 'E017-H3K4me2', 'E017-CTCF'),
 ('E017-H3K4me3', 'E017-H3K4me2', 'E017-H3K9ac'),
 ('E017-H3K4me3', 'E017-CTCF', 'E017-H3K9ac'),
 ('E017-H3K4me2', 'E017-CTCF', 'E017-H3K9ac'),
 ('E017-H3K4me3', 'E017-H3K4me2', 'E017-CTCF', 'E017-H3K9ac')]

In [11]:
if exclude_list[7] is not False:
    for excluded_input in exclude_list[7]:
        print(excluded_input)


E017-H3K4me3
E017-H3K9ac
